# Evaluation of $\alpha_{d}$ parameter variation
# Light-harvesting efficiency diatoms

#### Control: run 'mo5', $\alpha_{d}$ = 0.19, pacific hydrothermal vent masked

run 'mo4', $\alpha_{d}$ = 0.19, pacific hydrothermal vent NOT masked

run 'mo8', $\alpha_{d}$ = 0.18, pacific hydrothermal vent masked

run 'mo9', $\alpha_{d}$ = 0.20, pacific hydrothermal vent masked

In [ ]:
%autosave 5
%matplotlib inline

# To do
#### Include Datasets
isibhv/projects: OCCCI


In [ ]:
import pyfesom2 as pf
import math
import numpy as np
import matplotlib.pylab as plt
import matplotlib.cm as cm
from matplotlib import gridspec
from netCDF4 import Dataset
import time
from datetime import date
import cartopy.crs as ccrs
import pangaeapy.pandataset as pd

import skill_metrics as sm
from plot_Taylor_normalized import plt_Taylor_norm

!jupyter nbconvert --to=python F2R_sensitivity_toolbox.ipynb
import F2R_sensitivity_toolbox as stx

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# mesh ------------------------------------------------------------------------------------
meshpath = '/work/ollie/mozeisin/mesh/mesh_fesom2.0/core2_meanz'

# list of runs ------------------------------------------------------------------------------------
# first run speciefied is used as control run
resultpath = '/work/ollie/mozeisin/results/f2r1.2'
path_list = '/mo5', '/mo4', '/mo8', '/mo9'
resultpath = [resultpath +i for i in path_list]

# corresponding runid of FESOM runs
idlist = ('fesom', 'fesom', 'fesom', 'fesom') 
# see corresponding plotting labels below

# props of analysis ------------------------------------------------------------------------------------
# analysing fesom parameter
fesom_parameter = 'AlfaD'

first_year = 1951
last_year = 1957
years = [first_year, last_year]

depth=0

# props of plotting ------------------------------------------------------------------------------------
#pc = ccrs.PlateCarree()
#rob = ccrs.Robinson()
mapproj = 'pc'

# label used for plotting
figlabel = ('CTRL ($\\alpha_{d}$ = 0.19)', 'CTRL (without mask)', '$\\alpha_{d}$ = 0.18', '$\\alpha_{d}$ = 0.20') 

# labels used for saving output
savelabel = ('CTRL_AlfaD_19','CTRL_woMask_AlfaD_19','AlfaD_18','AlfaD_20')

# rows and columns for subplots
rowscol = (2,2)

# export of analysis ------------------------------------------------------------------------------------
savefig = True

savepath = '/home/ollie/mozeisin/evaluation/mo_files/F2R_sensitivity/'

today = date.today().strftime("_%Y_%m_%d")
htmlname     =  'F2R_sensitivity_' + fesom_parameter + '_' + today +'.html'
htmlpath = '/home/ollie/mozeisin/evaluation/mo_files/html/'

# initialization file specifications -----------------------------------------------------------
ncfileMaredat = "/home/ollie/mozeisin/obs/MarEDat/MarEDat20120716Diatoms.nc"

#obspath      = '/home/ollie/mseifert/py_fesom/CreateRunFiles/Initialization/Output/'

#ncfileAlk                = '/work/ollie/projects/MarESys/evaluation/GLODAPv2.2016b.TAlk.nc'
#ncfileDIC                = '/work/ollie/projects/MarESys/evaluation/GLODAPv2.2016b.TCO2.nc'
#ncfileDSi                = '/work/ollie/projects/MarESys/evaluation/woa13_all_i00_01.nc'
ncfileDIN                = '/work/ollie/projects/MarESys/evaluation/woa13_all_n00_01.nc'
#ncfileDO2                = '/work/ollie/projects/MarESys/evaluation/woa18_all_o00_01.nc'
#ncfileDFe                = '/work/ollie/projects/MarESys/evaluation/fe_pisces_opa_eq_init_3D.nc' 
#ncfileNPPTotalGlobal     = '/work/ollie/projects/MarESys/evaluation/'+'fesom.initial.mesh.diag.nc'
#ncfileCO2fluxTotalGlobal = '/work/ollie/projects/MarESys/evaluation/'+'fesom.initial.mesh.diag.nc'
#ncfileTemp               = '/work/ollie/projects/MarESys/evaluation/woa18_decav_t00_01.nc'
#ncfileSal                = '/work/ollie/projects/MarESys/evaluation/woa18_decav_s00_01.nc'
#ncfilepCO2               = '/work/ollie/projects/MarESys/evaluation/SOCATv6_tracks_gridded_monthly.nc'
#ncfileMLD                = '/work/ollie/projects/MarESys/evaluation/mld_DR003.nc'
matfileChlsurf           = '/work/ollie/projects/MarESys/evaluation/climatology_annual_chl_1deg_OCCCI_2012_2015.mat'
#txtfileCO2flux           = '/work/ollie/projects/MarESys/evaluation/CO2_flux2015_Takahashietal2009_original.txt'


#### quickly check input

In [ ]:
resultpath

In [ ]:
print('Period: {0} to {1}'.format(years[0],years[1]))

In [ ]:
# mesh initialization -----------------------------------------------------------
mesh = pf.load_mesh(meshpath)

# load FESOM mesh diag -------------------------------------------------------------------------------
meshdiag= meshpath +'fesom.mesh.diag.nc'
#!ncdump -h $meshdiag

diag = pf.get_meshdiag(mesh)
#print(diag)
#print(diag['Z']) # depth of layers
mesh_depths = diag['Z'].values

stx.check_plot_mesh(mesh, plot_globe = False)

# MarEDAT

In [ ]:
#!ncdump -h $ncfileMaredat

In [ ]:
# load maredat
lat_maredat, lon_maredat, maredat_layered = stx.maredat_load(ncfileMaredat)

In [ ]:
# maredat mean min max of 0-5m
print('MarEDAT mean min max at 0-5m:')
np.nanmean(maredat_layered[0]), np.nanmin(maredat_layered[0]), np.nanmax(maredat_layered[0])

In [ ]:
# overview plot: maredat
fig = stx.maredat_overview_plot(maredat_layered,lon_maredat, lat_maredat,mapproj)

# FESOM Diatom carbon

#### - load FESOM data for all specified runs
#### - plot comparison graph for one depth

In [ ]:
#NCfesom = resultpath[0] + '/DiaC.'+idlist[0]+'.1948.nc' # DiaC:units = "[mmol/m3]"
#!ncdump -h $NCfesom

units = 'Diatom intracell. Carbon [mg C m$^{{-3}}$]\n{0}-{1} at {2} m'.format(years[0],years[1],depth)
#units = 'Intracellular C [mg C m$^{-3}$]'

DiaC = [0]*len(idlist)

for i in range(len(idlist)):
    print(resultpath[i])
    DiaC[i] = pf.get_data(resultpath[i], "DiaC", years, mesh, runid=idlist[i], how="mean", compute=True, depth=depth)
    DiaC[i] = DiaC[i] * 12.01

fig = pf.plot(mesh, [data for data in DiaC], mapproj = mapproj, units=units, cmap = 'viridis',
              rowscol = rowscol, #(math.ceil(len(idlist)/3), 3), 
              figsize = (15,10),
              titles=[title for title in figlabel])

## differences
fig = pf.plot(mesh, [data-DiaC[0] for data in DiaC], mapproj = mapproj, units='MODEL - CTRL\n'+units, 
              rowscol = rowscol, #(math.ceil(len(idlist)/3), 3), 
              figsize = (15,10),
              #levels = (-2.5e-2,2.5e-2, 30), 
              cmap = 'RdBu',
              titles=[title for title in figlabel])

# FESOM comparison to MarEDat

In [ ]:
if False:
    print(np.shape(mesh.zlev)) # levels
    print(np.shape(mesh_depths)) # layers

    print(mesh.zlev)
    print(mesh_depths)

#### Interpolating FESOM depth-range-means to 1x1 grid

In [ ]:
mo5 = stx.fesom_to_maredat_levels(resultpath[0], idlist[0], years, mesh, lon_maredat, lat_maredat)
mo4 = stx.fesom_to_maredat_levels(resultpath[1], idlist[1], years, mesh, lon_maredat, lat_maredat)
mo6 = stx.fesom_to_maredat_levels(resultpath[2], idlist[2], years, mesh, lon_maredat, lat_maredat)
mo7 = stx.fesom_to_maredat_levels(resultpath[3], idlist[3], years, mesh, lon_maredat, lat_maredat)

#### Plot runs

In [ ]:
stx.fesom_plot_depths(mo5, figlabel[0], savelabel[0], lon_maredat, lat_maredat, 
                      mapproj, years, savefig=savefig, savepath=savepath)

In [ ]:
stx.fesom_plot_depths(mo4, figlabel[1], savelabel[1], lon_maredat, lat_maredat, 
                      mapproj, years, savefig=savefig, savepath=savepath)

In [ ]:
stx.fesom_plot_depths(mo6, figlabel[2], savelabel[2], lon_maredat, lat_maredat, 
                      mapproj, years, savefig=savefig, savepath=savepath)

In [ ]:
stx.fesom_plot_depths(mo7, figlabel[3], savelabel[3], lon_maredat, lat_maredat, 
                      mapproj, years, savefig=savefig, savepath=savepath)

# MarEDAT and FESOM side by side

In [ ]:
stx.plot_2cols(mo5,figlabel[0],savelabel[0], maredat_layered, lon_maredat, lat_maredat, mapproj, years, savefig, savepath)

In [ ]:
stx.plot_2cols(mo4,figlabel[1],savelabel[1], maredat_layered, lon_maredat, lat_maredat, mapproj, years, savefig, savepath)

In [ ]:
stx.plot_2cols(mo6,figlabel[2],savelabel[2], maredat_layered, lon_maredat, lat_maredat, mapproj, years, savefig, savepath)

In [ ]:
stx.plot_2cols(mo7,figlabel[3],savelabel[3], maredat_layered, lon_maredat, lat_maredat, mapproj, years, savefig, savepath)

# Prepare for statistics

### apply mask

In [ ]:
# mask model where MarEDAT not available
mo5_ma = stx.mask_model_with_maredat(mo5,maredat_layered)
mo4_ma = stx.mask_model_with_maredat(mo4,maredat_layered)
mo6_ma = stx.mask_model_with_maredat(mo6,maredat_layered)
mo7_ma = stx.mask_model_with_maredat(mo7,maredat_layered)

#### CONTROL of masking for 0-5 m 


In [ ]:
if True:
    fig, axes = plt.subplots(1,2, 
                             subplot_kw=dict(projection=ccrs.Robinson()),
                             gridspec_kw={'hspace': 0.001, 'wspace': 0.1},
                             figsize=(20,20))
    # 0-5 m maredat 
    m1 = axes[0]
    f1 = m1.pcolormesh(lon_maredat, lat_maredat, np.log10(maredat_layered[0]), 
                       transform = ccrs.PlateCarree(),
                        shading='flat', vmin=np.log10(0.001), vmax=np.log10(200), 
                        cmap=plt.cm.viridis)
    stx.mygrid(m1)
    m1.set_title('MarEDAT 0-5 m', fontsize=20)

    # FESOM 
    m1 = axes[1]
    f1 = m1.pcolormesh(lon_maredat, lat_maredat, np.log10(mo5_ma[0]), 
                       transform = ccrs.PlateCarree(),
                        shading='flat', vmin=np.log10(0.001), vmax=np.log10(200), 
                        cmap=plt.cm.viridis)
    stx.mygrid(m1)
    m1.set_title('FESOM 0-5 m', fontsize=20)

# Taylor statistics

In [ ]:
depth_range = [0,5,25,100,'bot']

In [ ]:
if False: #old
    fig, sdev, crmsd, ccoef = stx.plot_Taylor(maredat_layered[0],mo5_ma[0],
                        plot_depth = '0-5m')

In [ ]:
# print all runs at once
# mo5
fig, sdev, crmsd, ccoef_mo5 = stx.plot_Taylor_all_depths(maredat_layered,mo5_ma,depth_range)

In [ ]:
# mo4
fig, sdev, crmsd, ccoef_mo4 = stx.plot_Taylor_all_depths(maredat_layered,mo4_ma,depth_range)

In [ ]:
# mo6
fig, sdev, crmsd, ccoef_mo6 = stx.plot_Taylor_all_depths(maredat_layered,mo6_ma,depth_range)

In [ ]:
# mo7
fig, sdev, crmsd, ccoef_mo7 = stx.plot_Taylor_all_depths(maredat_layered,mo7_ma,depth_range)

# Scatter plots (as Miriam)

In [ ]:
stx.plot_scatter(mo5_ma, figlabel[0], savelabel[0], maredat_layered, years, savefig, savepath, ccoef_mo5)

In [ ]:
stx.plot_scatter(mo4_ma, figlabel[1], savelabel[1], maredat_layered, years, savefig, savepath, ccoef_mo4)


In [ ]:
stx.plot_scatter(mo6_ma, figlabel[2], savelabel[2], maredat_layered, years, savefig, savepath, ccoef_mo6)


In [ ]:
stx.plot_scatter(mo7_ma, figlabel[3], savelabel[3], maredat_layered, years, savefig, savepath, ccoef_mo7)


# Diatom silicate

In [ ]:
units = 'Diatom intracell. Silicate [mmol m$^{{-3}}$]\n{0}-{1}'.format(years[0],years[1])
#units = 'intracellular Si [mmol m$^{-3}$]'

DiaSi = [0]*len(idlist)

for i in range(len(idlist)):
    DiaSi[i] = pf.get_data(resultpath[i], "DiaSi", years, mesh, runid=idlist[i], how="mean", compute=True, depth=depth)

fig = pf.plot(mesh, [data for data in DiaSi], mapproj = mapproj, units=units, cmap = 'viridis',
              rowscol = rowscol, #(math.ceil(len(idlist)/3), 3), 
              figsize = (15,10),
              titles=[title for title in figlabel])

## differences
fig = pf.plot(mesh, [data-DiaSi[0] for data in DiaSi], mapproj = mapproj, units='MODEL - CTRL\n'+units, 
              rowscol = rowscol, #(math.ceil(len(idlist)/3), 3), 
              figsize = (15,10),
              #levels = (-2.5e-2,2.5e-2, 30), 
              cmap = 'RdBu',
              titles=[title for title in figlabel])

# Chl a

In [ ]:
# diatom chlorophyll
#!ncdump -h $NCfesom
depth = 0

DiaChl = [0]*len(idlist)

units ='Diatom Chl [mmol m$^{{-3}}$]\n{0}-{1} at {2} m'.format(years[0],years[1],depth)
#units = 'Chl.a [mmol m$^{-3}$]'

for i in range(len(idlist)):
    DiaChl[i] = pf.get_data(resultpath[i], "DiaChl", years, mesh, runid=idlist[i], how="mean", 
                            compute=True, depth=depth)

fig = pf.plot(mesh, [data for data in DiaChl], mapproj = mapproj, units=units, cmap = 'viridis',
              rowscol = rowscol, #(math.ceil(len(idlist)/3), 3), 
              figsize = (15,10),
              titles=[title for title in figlabel])

fig = pf.plot(mesh, [data-DiaChl[0] for data in DiaChl], mapproj = mapproj, units='MODEL - CTRL\n'+units, 
              rowscol = rowscol, #(math.ceil(len(idlist)/3), 3), 
              figsize = (15,10),
              #levels = (-2.5e-5,2.5e-5, 30), 
              cmap = 'RdBu',
              titles=[title for title in figlabel])

# Map of total Diatom NPP 

In [ ]:
#NCfesom = resultpath + '/NPPd.mo1.1948.nc'
#!ncdump -h $NCfesom

depth = 0

# 12.01* data --> mg C/m2/d

units ='Mean Diatom NPP [mg C m$^{{-2}}$ d$^{{-1}}$]\n{0}-{1} at {2} m'.format(years[0],years[1],depth)
#units = 'Mean NPP [mg C m$^{-2}$ d$^{-1}$]'

NPPd = [0]*len(idlist)

for i in range(len(idlist)):
    NPPd[i] = pf.get_data(resultpath[i], "NPPd", years, mesh, runid=idlist[i], how="mean", compute=True, depth=depth)
    NPPd[i] = NPPd[i] * 12.01

fig = pf.plot(mesh, [data for data in NPPd], mapproj = mapproj, units=units, cmap = 'viridis',
              rowscol = rowscol, #(math.ceil(len(idlist)/3), 3), 
              figsize = (15,10),
              titles=[title for title in figlabel])

## differences
fig = pf.plot(mesh, [data-NPPd[0] for data in NPPd], mapproj = mapproj, units='MODEL - CTRL\n'+units, 
              rowscol = rowscol, #(math.ceil(len(idlist)/3), 3), 
              figsize = (15,10),
              #levels = (-2.5e-2,2.5e-2, 30), 
              cmap = 'RdBu',
              titles=[title for title in figlabel])

# Map of nanophytoplankton NPP

In [ ]:
#NCfesom = resultpath + '/NPPn.mo1.1948.nc'
#!ncdump -h $NCfesom

# 12.01* data --> mg C/m2/d

units = 'Nanophytoplankton NPP [mg C m$^{{-2}}$ d$^{{-1}}$]\n{0}-{1} at {2} m'.format(years[0],years[1],depth)
#units = 'Mean NPP [mg C m$^{-2}$ d$^{-1}$]'

NPPn = [0]*len(idlist)

for i in range(len(idlist)):
    NPPn[i] = pf.get_data(resultpath[i], "NPPn", years, mesh, runid=idlist[i], how="mean", compute=True, depth=depth)
    NPPn[i] = NPPn[i] * 12.01

fig = pf.plot(mesh, [data for data in NPPn], mapproj = mapproj, units=units, cmap = 'viridis',
              rowscol = rowscol, #(math.ceil(len(idlist)/3), 3), 
              figsize = (15,10),
              titles=[title for title in figlabel])

## differences
fig = pf.plot(mesh, [data-NPPn[0] for data in NPPn], mapproj = mapproj, units='MODEL - CTRL\n'+units, 
              rowscol = rowscol, #(math.ceil(len(idlist)/3), 3), 
              figsize = (15,10),
              #levels = (-30,30, 30), 
              cmap = 'RdBu',
              titles=[title for title in figlabel])

# Total NPP

In [ ]:
units ='Total Net Primary Production [mg C m$^{{-2}}$ d$^{{-1}}$]\n{0}-{1}  at {2} m'.format(years[0],years[1],depth)
#units = 'Mean total NPP [mg C m$^{-2}$ d$^{-1}$]'

NPPtot = [NPPd[i] + NPPn[i] for i in range(len(idlist))]

fig = pf.plot(mesh, [data for data in NPPtot], mapproj = mapproj, units=units, cmap = 'viridis',
              rowscol = rowscol, #(math.ceil(len(idlist)/3), 3), 
              figsize = (15,10),
              titles=[title for title in figlabel])

## differences
fig = pf.plot(mesh, [data-NPPtot[0] for data in NPPtot], mapproj = mapproj, units='MODEL - CTRL\n'+units, 
              rowscol = rowscol, #(math.ceil(len(idlist)/3), 3), 
              figsize = (15,10),
              #levels = (-2.5e-1,2.5e-1, 30), 
              cmap = 'RdBu',
              titles=[title for title in figlabel])

# Nutrients

In [ ]:
from DIN_WOA_Comparison_python3 import DINcomp

In [ ]:
# check DIN comp to WOA FOR CONTROL RUN

#ctrl = DINcomp('fesom',resultpath[0],savepath,meshpath,ncfileDIN,first_year,last_year,mapproj=mapproj)

#### compare each FESOM run with WOA DIN

In [ ]:
# run DINcomp for each result
if True:
    for i in range(len(resultpath)):
        print('\n****************************** \nANALYSING {0} \n******************************'.format(resultpath[i]))
        fig = DINcomp(idlist[i],resultpath[i],savepath,meshpath,ncfileDIN,first_year,last_year,mapproj=mapproj)

## Transect

#### Atlantic :  Diatom Chl

In [ ]:
lon_start = -25
lat_start = -50
lon_end = -25
lat_end = 65
npoints   = 50
lonlat = pf.transect_get_lonlat(lon_start, lat_start, lon_end, lat_end, npoints)

pf.plot_transect_map(lonlat, mesh)

#### load FESOM 3d DiaC data

In [ ]:
#load 3D data
DiaC_3d = [0]*len(idlist)

for i in range(len(idlist)):
    print(resultpath[i])
    DiaC_3d_aux = pf.get_data(resultpath[i], "DiaC", years, mesh, runid=idlist[i], how="mean")
    DiaC_3d[i] = DiaC_3d_aux * 12.01


In [ ]:
print('Atlantic Transect, mean of {0}-{1}'.format(years[0],years[1]))

dist, transect_data = pf.get_transect(DiaC_3d[0],mesh,lonlat)
dist2, transect_data2 = pf.get_transect(DiaC_3d[1],mesh,lonlat)
dist3, transect_data3 = pf.get_transect(DiaC_3d[2],mesh,lonlat)
dist4, transect_data4 = pf.get_transect(DiaC_3d[3],mesh,lonlat)


pf.plot_xyz(mesh, 
            [transect_data,transect_data2,transect_data3,transect_data4],
            dist,
            maxdepth=500,
            ncols=2,
            xlabel='Distance',
            label='Diatom Chl',
            title=[l for l in figlabel],
            levels=np.linspace(0,40,41),#cmap=cm.hot_r,
            figsize=(15, 15))


# Other evaluation

## Bracher et al. (2019) PS106


In [ ]:
if False:
    # import from PANGAEA
    ds= pd.PanDataSet(899284)
    print(ds.title)
    # Chl a in µg/L

    PS106 = ds.data

    #print(np.shape(PS106), type(PS106))

    if False: # first overview plot
        fig_PS106 = PS106.plot(kind='scatter', x='Longitude', y='Latitude',
              c='Chl a diatoms', cmap=plt.get_cmap("jet"), colorbar=True,
              figsize = (10,7))

    #### plot with cartopy

    # defining projections
    og = ccrs.Orthographic(0,90)
    pc = ccrs.PlateCarree()

    PS106_Chla_dia = PS106['Chl a diatoms']
    lat, lon = PS106['Latitude'], PS106['Longitude']

    fig = plt.figure(figsize=(20, 16))
    ax = plt.axes(projection=og)
    ax.coastlines()
    ax.gridlines()
    plt.scatter(lon, lat, transform = pc, 
        c=PS106_Chla_dia, cmap=plt.get_cmap("jet"))
    ax.set_extent([-20, 50, 60, 90])
    ax.stock_img()
    plt.xlabel('longitude')
    plt.ylabel('latitude')
    plt.title('PS106 Diatom Chlorophyll a  (Bracher et al., 2019)')
    plt.colorbar(label='Diatom Chl.a [µg/L]')
    #plt.clim(0, 10)

## Evaluation scripts from pyfesom 2

#### Diatom Chla integrals

In [ ]:
DiaChl = pf.get_data(resultpath[0], 'DiaChl', years, mesh, runid=idlist[0], how=None, compute=False)

In [ ]:
#pf.volmean_data?
DiaChl_volmean = pf.volmean_data(DiaChl, mesh)

In [ ]:
DiaChl_volmean.plot()

# Export to HTML

In [ ]:
%autosave 2

time.sleep(15)

# convert without input (code)
!jupyter nbconvert MZ_F2R_sensitivity_runs_varAlfaD.ipynb --output-dir=$htmlpath --output $htmlname --to html --no-input